# URL clustering based on similarity

In [54]:
import matplotlib as plt
import numpy as np
import pandas as pd
import seaborn as sns

sns.set()

In [55]:
urls_df = pd.read_csv("datasets/kaggle_siddharta_malicious_benign.csv", delimiter=",")

urls_df.head()

,url,type
0,br-icloud.com.br,phishing
1,mp3raid.com/music/krizz_kaliko.html,benign
2,bopsecrets.org/rexroth/cr/1.htm,benign
3,http://www.garage-pirenne.be/index.php?option=...,defacement
4,http://adventure-nicaragua.net/index.php?optio...,defacement


In [68]:
# Create OneHotEncoded features from type

from sklearn.preprocessing import OneHotEncoder

ohenc = OneHotEncoder(sparse_output=False)
type_ohenc = pd.DataFrame(
    ohenc.fit_transform(urls_df["type"].values.reshape(-1, 1)),
    columns=ohenc.categories_[0],
).astype(bool)

# URLs_transformed df
urls_tdf = pd.concat([urls_df, type_ohenc], axis=1)

In [69]:
urls_tdf["malicious"] = ~urls_tdf["benign"]
urls_tdf.head()

,url,type,benign,defacement,malware,phishing,malicious
0,br-icloud.com.br,phishing,False,False,False,True,True
1,mp3raid.com/music/krizz_kaliko.html,benign,True,False,False,False,False
2,bopsecrets.org/rexroth/cr/1.htm,benign,True,False,False,False,False
3,http://www.garage-pirenne.be/index.php?option=...,defacement,False,True,False,False,True
4,http://adventure-nicaragua.net/index.php?optio...,defacement,False,True,False,False,True


In [ ]:
urls_tdf["benign"].astype(bool)

In [67]:
urls_tdf

,url,type,0,1,2,3
0,br-icloud.com.br,phishing,False,False,False,True
1,mp3raid.com/music/krizz_kaliko.html,benign,True,False,False,False
2,bopsecrets.org/rexroth/cr/1.htm,benign,True,False,False,False
3,http://www.garage-pirenne.be/index.php?option=...,defacement,False,True,False,False
4,http://adventure-nicaragua.net/index.php?optio...,defacement,False,True,False,False
...,...,...,...,...,...,...
651186,xbox360.ign.com/objects/850/850402.html,phishing,False,False,False,True
651187,games.teamxbox.com/xbox-360/1860/Dead-Space/,phishing,False,False,False,True
651188,www.gamespot.com/xbox360/action/deadspace/,phishing,False,False,False,True
651189,en.wikipedia.org/wiki/Dead_Space_(video_game),phishing,False,False,False,True
